## Deliverable 2. Create a Customer Travel Destinations Map.

In [31]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [32]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Omsukchan,RU,62.5333,155.8000,38.37,81,78,5.32,broken clouds
1,1,Los Llanos De Aridane,ES,28.6585,-17.9182,67.75,94,75,11.27,broken clouds
2,2,Garliava,LT,54.8167,23.8667,71.58,73,18,8.10,few clouds
3,3,Yellowknife,CA,62.4560,-114.3525,57.72,62,75,11.50,light rain
4,4,Carnarvon,AU,-24.8667,113.6333,55.47,88,91,9.22,overcast clouds


In [33]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [34]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Garliava,LT,54.8167,23.8667,71.58,73,18,8.10,few clouds
6,6,Kavaratti,IN,10.5669,72.6420,83.98,77,98,17.96,light rain
12,12,Puerto Ayora,EC,-0.7393,-90.3518,75.54,89,67,5.99,broken clouds
13,13,Torbay,CA,47.6666,-52.7314,72.37,49,20,16.11,few clouds
17,17,Tricase,IT,39.9304,18.3553,76.35,49,0,8.01,clear sky
20,20,Albany,US,42.6001,-73.9662,73.72,75,99,1.99,overcast clouds
21,21,Kirakira,SB,-10.4544,161.9205,79.23,88,100,8.66,moderate rain
25,25,Atuona,PF,-9.8000,-139.0333,78.93,81,52,18.68,light rain
26,26,Soyo,AO,-6.1349,12.3689,82.45,78,15,10.40,few clouds
28,28,Pontianak,ID,-0.0333,109.3333,80.58,100,51,0.92,broken clouds


In [35]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                278
City                   278
Country                275
Lat                    278
Lng                    278
Max Temp               278
Humidity               278
Cloudiness             278
Wind Speed             278
Current Description    278
dtype: int64

In [37]:
#4b. Drop any empty rows from the datadrame if any
clean_df = preferred_cities_df.dropna(how='any')

In [38]:
# 4c. Check the datafram again for empty rows
clean_df.count()

City_ID                275
City                   275
Country                275
Lat                    275
Lng                    275
Max Temp               275
Humidity               275
Cloudiness             275
Wind Speed             275
Current Description    275
dtype: int64

In [39]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Garliava,LT,71.58,few clouds,54.8167,23.8667,
6,Kavaratti,IN,83.98,light rain,10.5669,72.6420,
12,Puerto Ayora,EC,75.54,broken clouds,-0.7393,-90.3518,
13,Torbay,CA,72.37,few clouds,47.6666,-52.7314,
17,Tricase,IT,76.35,clear sky,39.9304,18.3553,
20,Albany,US,73.72,overcast clouds,42.6001,-73.9662,
21,Kirakira,SB,79.23,moderate rain,-10.4544,161.9205,
25,Atuona,PF,78.93,light rain,-9.8000,-139.0333,
26,Soyo,AO,82.45,few clouds,-6.1349,12.3689,
28,Pontianak,ID,80.58,broken clouds,-0.0333,109.3333,


In [40]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [45]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Garliava,LT,71.58,few clouds,54.8167,23.8667,"Armenia, cafe-motel, TIR parking"
6,Kavaratti,IN,83.98,light rain,10.5669,72.6420,Hotels in Lakshadweep Islands
12,Puerto Ayora,EC,75.54,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
13,Torbay,CA,72.37,few clouds,47.6666,-52.7314,Gallows Cove
17,Tricase,IT,76.35,clear sky,39.9304,18.3553,Hotel Adriatico
...,...,...,...,...,...,...,...
686,Calama,CL,70.29,clear sky,-22.4667,-68.9333,Diego De Almagro Calama
687,Weligama,LK,79.39,broken clouds,5.9667,80.4167,Weligama Bay Resort
689,Jasper,US,77.85,clear sky,37.2001,-94.3502,Guest House Motel
691,Dawei,MM,81.36,broken clouds,14.0833,98.2000,Hotel Dawei


In [46]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [49]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [50]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))